### final

In [ ]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.4f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# 인스타그램 게시글 수 가져오기
sql = 'select * from m1.instagram_emd_cnt'
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_instagram = pd.DataFrame(rows)
df_instagram.columns = ['emd_nm','emd_cd','cnt','base_ym']
df_instagram['cnt'] = df_instagram['cnt'].astype(float)

# 거주 인구 (법정동 단위)
sql = 'select emd_cd, l_tot from m2.cremao_local_population'
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_cremao_local_population = pd.DataFrame(rows)
df_cremao_local_population.columns = ['emd_cd','l_tot']
df_cremao_local_population['emd_cd'] = df_cremao_local_population['emd_cd'] + '00'
df_cremao_local_population['l_tot']  = df_cremao_local_population['l_tot'].astype(float)


# 데이터 프레임 병합
df = pd.merge(df_instagram, df_cremao_local_population, on='emd_cd', how='left')

# 인구가 0명이면 제외
idx_tot = df[df['l_tot']==0].index
df = df.drop(idx_tot).reset_index(drop=True)

# 게시글 수 없는 지역 제외
idx_nm = df[(df['emd_nm']=='구호동') | (df['emd_nm']=='법성면') | (df['emd_nm']=='대소원면')].index
df = df.drop(idx_nm).reset_index(drop=True)

df['calculate'] = (df['l_tot'] / df['cnt']) * 100
df = df[(df['calculate']>=5) & (df['calculate']<=350)].reset_index(drop=True)
df = df[['emd_nm','emd_cd','calculate','base_ym']]


### 2. DB INSERT
import datetime

table = 'm2.datadam_floating_population_per_sns'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    # drop & create
    cursor.execute(
    """
    drop table if exists m2.datadam_floating_population_per_sns;
    create table m2.datadam_floating_population_per_sns (    
    emd_nm varchar(50)
    , emd_cd varchar(10)
    , calculate double precision
    , base_ym varchar(6)
    )
    distkey(emd_cd)
    compound sortkey(emd_nm, base_ym);
    commit;
    """
    )
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

In [156]:
DF

,emd_nm,emd_cd,calculate,base_ym
0,가수동,4137012100,174.8210,202210
1,가수원동,3017011400,38.6348,202210
2,가야동,2623011000,28.1415,202210
3,가오동,3011010500,169.8468,202210
4,가음동,4812313100,108.8208,202210
...,...,...,...,...
1747,후포면,4793038000,7.0110,202210
1748,휘경동,1123010900,91.1954,202210
1749,휴천동,4721010400,7.8518,202210
1750,흑석동,2920012400,15.9786,202210


### test

In [1]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.4f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

In [2]:
# 인스타그램 게시글 수 가져오기
sql = 'select * from m1.instagram_emd_cnt'
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_instagram = pd.DataFrame(rows)
df_instagram.columns = ['emd_nm','emd_cd','cnt','base_ym']
df_instagram['cnt'] = df_instagram['cnt'].astype(float)

In [4]:
df_instagram

,emd_nm,emd_cd,cnt,base_ym
0,가남읍,4167025000,5598,202210
1,가대동,3120010400,4728,202210
2,가락동,1171010700,4146,202210
3,가례면,4872031000,2385,202210
4,가북면,4888041000,2731,202210
...,...,...,...,...
3963,흑산면,4691036000,3135,202210
3964,흑석동,2920012400,43771,202210
3965,흑암동,4518012300,48,202210
3966,흥각동,4725011700,2015,202210


In [44]:
# 거주 인구 (법정동 단위)
sql = 'select emd_cd, l_tot from m2.cremao_local_population'
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_cremao_local_population = pd.DataFrame(rows)
df_cremao_local_population.columns = ['emd_cd','l_tot']
df_cremao_local_population['emd_cd'] = df_cremao_local_population['emd_cd'] + '00'
df_cremao_local_population['l_tot']  = df_cremao_local_population['l_tot'].astype(float)

In [45]:
df_cremao_local_population

,emd_cd,l_tot
0,1111010100,3214.0000
1,1111010200,14541.0000
2,1111010300,989.0000
3,1111010400,11195.0000
4,1111010500,7929.0000
...,...,...
5049,5013025000,203.0000
5050,5013025300,78.0000
5051,5013025900,120.0000
5052,5013031000,88.0000


In [143]:
# 데이터 프레임 병합
df = pd.merge(df_instagram, df_cremao_local_population, on='emd_cd', how='left')

# 인구가 0명이면 제외
idx_tot = df[df['l_tot']==0].index
df = df.drop(idx_tot).reset_index(drop=True)

# 게시글 수 없는 지역 제외
idx_nm = df[(df['emd_nm']=='구호동') | (df['emd_nm']=='법성면') | (df['emd_nm']=='대소원면')].index
df = df.drop(idx_nm).reset_index(drop=True)

df['calculate'] = (df['l_tot'] / df['cnt']) * 100

In [144]:
df

,emd_nm,emd_cd,cnt,base_ym,l_tot,calculate
0,가남읍,4167025000,5598.0000,202210,190.0000,3.3941
1,가대동,3120010400,4728.0000,202210,28.0000,0.5922
2,가락동,1171010700,4146.0000,202210,21684.0000,523.0101
3,가례면,4872031000,2385.0000,202210,31.0000,1.2998
4,가북면,4888041000,2731.0000,202210,11.0000,0.4028
...,...,...,...,...,...,...
3901,흑산면,4691036000,3135.0000,202210,38.0000,1.2121
3902,흑석동,2920012400,43771.0000,202210,6994.0000,15.9786
3903,흑암동,4518012300,48.0000,202210,80.0000,166.6667
3904,흥각동,4725011700,2015.0000,202210,100.0000,4.9628


In [118]:
DF = df[(df['calculate']>=5) & (df['calculate']<=350)].reset_index(drop=True)

In [147]:
# 평균
df['calculate'].mean(axis=0)

1009.6299890828795

In [146]:
# 평균
DF['calculate'].mean(axis=0)

78.99992854813614

In [131]:
# 표준편차
DF['calculate'].std(axis=0)

84.54456805529831

In [135]:
# 표준편차 2
np.std(DF['calculate'], axis=0)

84.52043659539322

In [136]:
# 표준편차 3
DF['calculate'].values.std(axis=0)

84.52043659539322

In [120]:
DF

,emd_nm,emd_cd,cnt,base_ym,l_tot,calculate
0,가수동,4137012100,2514.0000,202210,4395.0000,174.8210
1,가수원동,3017011400,17536.0000,202210,6775.0000,38.6348
2,가야동,2623011000,36416.0000,202210,10248.0000,28.1415
3,가오동,3011010500,4046.0000,202210,6872.0000,169.8468
4,가음동,4812313100,10600.0000,202210,11535.0000,108.8208
...,...,...,...,...,...,...
1747,후포면,4793038000,4279.0000,202210,300.0000,7.0110
1748,휘경동,1123010900,26827.0000,202210,24465.0000,91.1954
1749,휴천동,4721010400,34591.0000,202210,2716.0000,7.8518
1750,흑석동,2920012400,43771.0000,202210,6994.0000,15.9786


In [137]:
DF['calculate'].describe()

count   1752.0000
mean      78.9999
std       84.5446
min        5.0275
25%       13.8497
50%       42.1264
75%      121.9407
max      349.7438
Name: calculate, dtype: float64

In [138]:
df.sort_values('calculate',ascending=True)

,emd_nm,emd_cd,cnt,base_ym,l_tot,calculate
2874,서면,4283031000,13417268.0000,202210,9.0000,0.0001
2538,망상동,4217012900,13442596.0000,202210,86.0000,0.0006
333,금고동,3020014800,99317.0000,202210,1.0000,0.0010
1857,청풍동,2917012200,2929119.0000,202210,47.0000,0.0016
1577,이인면,4415031000,1769228.0000,202210,43.0000,0.0024
...,...,...,...,...,...,...
2067,화곡동,1150010300,41.0000,202210,31045.0000,75719.5122
1339,영등포동1가,1156010200,63.0000,202210,52659.0000,83585.7143
869,부용동2가,2614010800,39.0000,202210,34154.0000,87574.3590
2349,당산동5가,1156011500,31.0000,202210,39584.0000,127690.3226


In [139]:
df = DF

In [ ]:
### 3. DB INSERT
import datetime

table = 'm2.datadam_f_tot_per_sns'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

### 참고

In [78]:
# 법정동별 유동인구 
sql = '''
    select substring (pnu, 0, 11), sum(f_tot)
      from m2.cremao_land_floating_population
     group by substring (pnu, 0, 11)
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_f_tot = pd.DataFrame(rows)
df_f_tot.columns = ['emd_cd','f_tot']

In [79]:
df_f_tot

,emd_cd,f_tot
0,1111010100,5717165
1,1111010200,4621997
2,1111010300,1106859
3,1111010600,39911228
4,1111011000,51802938
...,...,...
18597,4889039030,152
18598,4889041026,535
18599,4889041030,777
18600,4889042022,126


In [84]:
# 데이터 프레임 병합
df = pd.merge(df_instagram, df_f_tot, on='emd_cd', how='left')
DF = df[~df['f_tot'].isnull()].reset_index(drop=True)
DF = DF.drop([1528,1661]) # inf 값 제거
DF['cnt'] = DF['cnt'].astype(float) # 계산할 컬럼 타입 변경
DF['calculate'] = (DF['f_tot'] / DF['cnt']) * 100 # 비율 계산식

In [85]:
DF.sort_values('calculate', ascending=True)

,emd_nm,emd_cd,cnt,base_ym,f_tot,calculate
1524,구완동,3014012100,8463.0000,202210,15.0000,0.1772
1297,청풍동,2917012200,2929119.0000,202210,9049.0000,0.3089
2405,이기동,4217012500,5149.0000,202210,16.0000,0.3107
1290,청소면,4418034000,240164.0000,202210,1200.0000,0.4997
225,금고동,3020014800,99317.0000,202210,573.0000,0.5769
...,...,...,...,...,...,...
2335,우동,2635010500,38.0000,202210,680687737.0000,1791283518.4211
2158,신월동,1147010300,36.0000,202210,726230988.0000,2017308300.0000
204,구로동,1153010200,56.0000,202210,1437855067.0000,2567598333.9286
1989,상계동,1135010500,27.0000,202210,1333539468.0000,4939035066.6667


In [86]:
DF['calculate'].describe()

count         2740.0000
mean      15983116.5896
std      174513899.8182
min              0.1772
25%          14784.8652
50%         176029.2797
75%        1068406.2770
max     5694416058.5366
Name: calculate, dtype: float64